In [11]:
def load_mnist(path, kind='train'):
    import os
    import gzip
    import numpy as np
 
    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)
 
    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)
 
    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 28, 28)
 
    return images, labels

In [12]:
x_train, y_train = load_mnist('data/fashion', kind='train')
x_test, y_test = load_mnist('data/fashion', kind='t10k')

print(x_train.shape)

(60000, 28, 28)


In [13]:
import tensorflow as tf
import numpy as np

class MNISTLoader():
    def __init__(self):
        self.train_data, self.train_flag = load_mnist('data/fashion', kind='train')
        self.test_data, self.test_flag = load_mnist('data/fashion', kind='train')
        
        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis = -1)
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis = -1)
        
        self.train_flag = self.train_flag.astype(np.int32)
        self.test_flag = self.test_flag.astype(np.int32)
        
        self.train_num = self.train_data.shape[0]
        self.test_num = self.test_data.shape[0]
        
        print(self.train_num, self.train_data.shape)
        
    def get_batch(self, batch_size):
        index = np.random.randint(0, self.train_data.shape[0], batch_size)
        train_data_batch = tf.image.resize_with_pad(self.train_data[index, :], 224, 224)
        return train_data_batch, self.train_flag[index]

In [14]:
data = MNISTLoader()
X, y = data.get_batch(50)
print(X.shape)

60000 (60000, 28, 28, 1)
(50, 224, 224, 1)


In [15]:
class CNN(tf.keras.Model):
    def __init__(self):
        super().__init__()
        
        self.conv1 = tf.keras.layers.Conv2D(
        filters = 32,
        kernel_size = [5,5],
        padding = "same",
        activation = tf.nn.relu)
        
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size = [2, 2], strides = 2)
        
        self.conv2 = tf.keras.layers.Conv2D(
        filters = 64,
        kernel_size = [5, 5],
        padding = "same",
        activation = tf.nn.relu)
        
        self.pool2 = tf.keras.layers.MaxPool2D(pool_size = [2, 2], strides = 2)
        
        self.flatten = tf.keras.layers.Reshape(target_shape = (7 * 7 * 64, ))
        
        self.dense1 = tf.keras.layers.Dense(units = 1024, activation = tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units = 10)
        
        
    def call(self, input):
        x = self.conv1(input)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        output = tf.nn.softmax(x)
        return output

In [16]:
class LeNet(tf.keras.Model):
    def __init__(self):
        super().__init__()
        
        self.conv1 = tf.keras.layers.Conv2D(
        filters = 6,
        kernel_size = [5,5],
        padding = 'same',
        activation = tf.nn.sigmoid)
        
        self.pool1 = tf.keras.layers.AvgPool2D(pool_size = [2, 2], strides = 2)
        
        self.conv2 = tf.keras.layers.Conv2D(
        filters = 16,
        kernel_size = [5, 5],
        padding = "same",
        activation = tf.nn.sigmoid)
        
        self.pool2 = tf.keras.layers.AvgPool2D(pool_size = [2, 2], strides = 2)
        
        self.flatten = tf.keras.layers.Reshape(target_shape = (7 * 7 * 16, ))
        
        self.dense1 = tf.keras.layers.Dense(units = 120, activation = tf.nn.sigmoid)
        self.dense2 = tf.keras.layers.Dense(units = 84, activation = tf.nn.sigmoid)
        self.dense3 = tf.keras.layers.Dense(units = 10)
        
        
    def call(self, input):
        print(input.shape)
        x = self.conv1(input)
        print(x.shape)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        output = tf.nn.softmax(x)
        return output

In [17]:
class AlexNet(tf.keras.Model):
    def __init__(self):
        super().__init__()
        
        self.conv1 = tf.keras.layers.Conv2D(
        filters = 96,
        kernel_size = [11,11],
        strides = 4,
        activation = tf.nn.relu)   # [:, 18, 18, 96]
        
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size = [3, 3], strides = 2) # [:, 8, 8, 96]
        
        self.conv2 = tf.keras.layers.Conv2D(
        filters = 256,
        kernel_size = [5, 5],
        padding = "same",
        activation = tf.nn.relu)   # [:, 8, 8, 256]
        
        self.pool2 = tf.keras.layers.MaxPool2D(pool_size = [3, 3], strides = 2) # [:, 3, 3, 256]
        
        self.conv3 = tf.keras.layers.Conv2D(
        filters = 384,
        kernel_size = [3, 3],
        padding = "same",
        activation = tf.nn.relu)   # [:, 3, 3, 384]
        
        self.conv4 = tf.keras.layers.Conv2D(
        filters = 384,
        kernel_size = [3, 3],
        padding = "same",
        activation = tf.nn.relu)
        
        self.conv5 = tf.keras.layers.Conv2D(
        filters = 256,
        kernel_size = [3, 3],
        padding = "same",
        activation = tf.nn.relu)
        
        self.pool3 = tf.keras.layers.MaxPool2D(pool_size = [3, 3], strides = 2)
        
        self.flatten = tf.keras.layers.Flatten()
        
        self.dropout = tf.keras.layers.Dropout(0.5)
        
        self.dense1 = tf.keras.layers.Dense(units = 4096, activation = tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units = 4096, activation = tf.nn.relu)
        self.dense3 = tf.keras.layers.Dense(units = 10)
        
        
    def call(self, input):
        x = self.conv1(input)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.pool3(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dropout(x)
        x = self.dense2(x)
        x = self.dropout(x)
        x = self.dense3(x)
        output = tf.nn.softmax(x)
        return output

In [18]:
class VGG(tf.keras.Model):
    def __init__(self):
        super().__init__()
        
        self.conv1 = tf.keras.layers.Conv2D(
        filters = 64,
        kernel_size = [3,3],
        padding = "same",
        activation = tf.nn.relu)   # [:, 18, 18, 96]
        
        self.pool = tf.keras.layers.MaxPool2D(pool_size = [2,2], strides = 2) # [:, 8, 8, 96]
        
        self.conv2 = tf.keras.layers.Conv2D(
        filters = 128,
        kernel_size = [3, 3],
        padding = "same",
        activation = tf.nn.relu)   # [:, 8, 8, 256]
        
        self.conv3 = tf.keras.layers.Conv2D(
        filters = 256,
        kernel_size = [3, 3],
        padding = "same",
        activation = tf.nn.relu)   # [:, 3, 3, 384]
        
        self.conv4 = tf.keras.layers.Conv2D(
        filters = 256,
        kernel_size = [3, 3],
        padding = "same",
        activation = tf.nn.relu)   # [:, 3, 3, 384]
        
        self.conv5 = tf.keras.layers.Conv2D(
        filters = 512,
        kernel_size = [3, 3],
        padding = "same",
        activation = tf.nn.relu)
        
        self.conv6 = tf.keras.layers.Conv2D(
        filters = 512,
        kernel_size = [3, 3],
        padding = "same",
        activation = tf.nn.relu)
        
        self.conv7 = tf.keras.layers.Conv2D(
        filters = 512,
        kernel_size = [3, 3],
        padding = "same",
        activation = tf.nn.relu)
        
        self.conv8 = tf.keras.layers.Conv2D(
        filters = 512,
        kernel_size = [3, 3],
        padding = "same",
        activation = tf.nn.relu)
        
        self.flatten = tf.keras.layers.Flatten()
        
        self.dropout = tf.keras.layers.Dropout(0.5)
        
        self.dense1 = tf.keras.layers.Dense(units = 4096, activation = tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units = 4096, activation = tf.nn.relu)
        self.dense3 = tf.keras.layers.Dense(units = 10)
        
        
    def call(self, input):
        x = self.conv1(input)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.pool(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.pool(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.pool(x)
        x = self.conv7(x)
        x = self.conv8(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dropout(x)
        x = self.dense2(x)
        x = self.dropout(x)
        x = self.dense3(x)
        output = tf.nn.softmax(x)
        return output

In [22]:
import time

start =time.time()
num_epochs = 10
batch_size = 50
learning_rate = 0.001

filename = 'train_test_VGG.txt'

end = time.time()

module = VGG()
data_loader = MNISTLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate)

num_batchs = int(data_loader.train_num / batch_size * num_epochs)
for batch_index in range(num_batchs):
    X, y = data_loader.get_batch(batch_size)
    with tf.GradientTape() as tape:
        y_pred = module(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true = y, y_pred = y_pred)
        loss = tf.reduce_mean(loss)
        with open(filename,'a') as file_object:
            if (batch_index % 1 == 0) :
                file_object.write("batch: %d, loss: %f\n" %(batch_index, loss.numpy()))
                print("batch: %d, loss: %f" %(batch_index, loss.numpy()))
        
    grads = tape.gradient(loss, module.variables)
    optimizer.apply_gradients(grads_and_vars = zip(grads, module.variables))
end = time.time()
print('Running time: %d m  %f s, %f examples/sec'%((end-start) / 60,
                                                   (end-start) % 60, num_batchs*batch_size/(end-start)))

60000 (60000, 28, 28, 1)
batch: 0, loss: 2.302439
batch: 1, loss: 3.026785
batch: 2, loss: 2.302477
batch: 3, loss: 2.299973
batch: 4, loss: 2.302078
batch: 5, loss: 2.281440
batch: 6, loss: 2.393549
batch: 7, loss: 2.273713


KeyboardInterrupt: 

In [11]:
tf.saved_model.save(module, 'VGG')

INFO:tensorflow:Assets written to: VGG/assets


In [6]:
import time
module = tf.saved_model.load('VGG')
data_loader = MNISTLoader()
batch_size = 50
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batchs = int(data_loader.test_num / batch_size)
start = time.time()
for batch_index in range(num_batchs):
    start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
    y_pred = module(tf.image.resize_with_pad(data_loader.test_data[start_index:end_index], 224, 224))
    with open('test_VGG.txt', 'a') as file_object:
        if (batch_index % 100 == 0) :
            print ("num_batchs: %d, batch_index: %d" % (num_batchs, batch_index))
            file_object.write('num_batchs: %d, batc_index: %d\n' %(num_batchs, batch_index) )
    
sparse_categorical_accuracy.update_state(y_true = data_loader.test_flag[start_index:end_index], y_pred = y_pred)
end = time.time()
with open('test_VGG.txt', 'a') as file_object:
    print(data_loader.test_flag.shape)
    print("test saccuracy: %f" % sparse_categorical_accuracy.result())
    print("Running time: %d m  %f s" % ((end-start) / 60, (end-start) % 60))
    file_object.write("test saccuracy: %f" % sparse_categorical_accuracy.result())
    file_object.write("Running time: %d m  %f s" % ((end-start) / 60, (end-start) % 60))

60000 (60000, 28, 28, 1)


2022-06-18 22:31:15.573576: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-06-18 22:31:15.590455: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2595120000 Hz


num_batchs: 1200, batch_index: 0
num_batchs: 1200, batch_index: 100
num_batchs: 1200, batch_index: 200
num_batchs: 1200, batch_index: 300
num_batchs: 1200, batch_index: 400
num_batchs: 1200, batch_index: 500
num_batchs: 1200, batch_index: 600
num_batchs: 1200, batch_index: 700
num_batchs: 1200, batch_index: 800
num_batchs: 1200, batch_index: 900
num_batchs: 1200, batch_index: 1000
num_batchs: 1200, batch_index: 1100
(60000,)
test saccuracy: 0.960000
Running time: 163 m  42.043810 s
